## More automated approach to training architectures

### Setting up the imports/project path

In [6]:
# Ensure notebook sees necessary paths
import os, sys
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.insert(0, project_root)

In [7]:
# Imports
from DataObjects.DataLoader import DataLoader
from Architectures.DifussionModel import DiffusionModel
from Trainers.DiffusionTrainer import TrainingConfig

# Noise schedulers
from diffusers import DDPMScheduler
from diffusers.optimization import get_cosine_schedule_with_warmup
from diffusers import DDPMPipeline

# Torch files
import torch
import torch.nn.functional as F

from Utils.utils import make_grid, evaluate

from Architectures.DifussionModel import train_loop

In [8]:
#Set up the correct device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


### Setup training objects for diffusion

In [4]:
# loading the config
config = TrainingConfig().from_json("../configs/config1.json")

In [9]:
# Load the data
Data = DataLoader(config.data_dir, batch_size=config.train_batch_size, shuffle=True, fraction = config.fraction, raw = True, dim_shape=config.image_size)
print(f"Number of data batches: {len(Data)}")

train_dataloader = torch.utils.data.DataLoader(Data, batch_size=config.train_batch_size, shuffle=True)

Number of data batches: 78


In [ ]:
noise_scheduler = DDPMScheduler(num_train_timesteps=config.num_train_timesteps, beta_start=config.beta_start, beta_end=config.beta_end, beta_schedule=config.beta_schedule, variance_type=config.variance_type)
model = DiffusionModel.return_custom_arch(config)
optimizer = torch.optim.AdamW(model.parameters(), lr=config.learning_rate)

In [ ]:
lr_scheduler = get_cosine_schedule_with_warmup(
    optimizer=optimizer,
    num_warmup_steps=config.lr_warmup_steps,
    num_training_steps=(len(Data) * config.num_epochs * config.train_batch_size),
)

In [ ]:
## Launch the training
from accelerate import notebook_launcher
args = (config, model, noise_scheduler, optimizer, Data, lr_scheduler, device)

notebook_launcher(train_loop, args, num_processes=1)

Launching training on one GPU.


d:\Pulpit\DeepLearning\DeepLearning\Project_III\venv\Lib\site-packages\accelerate\accelerator.py:498: UserWarning: `log_with=tensorboard` was passed but no supported trackers are currently installed.
  warnings.warn(f"`log_with={log_with}` was passed but no supported trackers are currently installed.")
Epoch 0:   0%|          | 0/95 [00:00<?, ?it/s]d:\Pulpit\DeepLearning\DeepLearning\Project_III\venv\Lib\site-packages\diffusers\configuration_utils.py:141: FutureWarning: Accessing config attribute `num_train_timesteps` directly via 'DDPMScheduler' object attribute is deprecated. Please access 'num_train_timesteps' over 'DDPMScheduler's config object instead, e.g. 'scheduler.config.num_train_timesteps'.
  deprecate("direct config name access", "1.0.0", deprecation_message, standard_warn=False)
Epoch 74: 100%|██████████| 95/95 [02:06<00:00,  1.33s/it, loss=0.00917, lr=9.98e-5, step=7124]
